# Ilab 算法竞赛

In [1]:
print("Hello ilab !!!")

Hello ilab !!!


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms

import torch.optim as optim
import torchvision.models as models

import PIL.Image as Image
import numpy as np
import pandas as pd
import os
import csv

In [3]:
!wget -o /home/ilab/YOLOV5s.zip https://mxjmxj.oss-cn-hangzhou.aliyuncs.com/YOLOV5s.zip

In [4]:
!wget -o /home/ilab/best_155.pt https://mxjmxj.oss-cn-hangzhou.aliyuncs.com/best_155.pt

In [5]:
import zipfile
with zipfile.ZipFile("YOLOV5s.zip") as zf:
    zf.extractall()

In [6]:
!pip install pyaml -i https://pypi.douban.com/simple/

Looking in indexes: https://pypi.douban.com/simple/


In [7]:
!wget -o /home/best.pt https://xujiayi1.oss-cn-beijing.aliyuncs.com/best.pt

In [8]:
!ls

best_155.pt	 data	     models	       train.py        YOLOV5s.zip.1
best_155.pt.1	 detect.py   __pycache__       tutorial.ipynb  YOLOV5s.zip.2
best_155.pt.2	 export.py   README.md	       utils
best.pt		 fall3.yaml  requirements.txt  val.py
best.pt.1	 hubconf.py  runs	       work
CONTRIBUTING.md  LICENSE     setup.cfg	       YOLOV5s.zip


In [10]:
!ls data

Argoverse.yaml	      hyps	       scripts	      xView.yaml
coco128.yaml	      img3	       SKU-110K.yaml
coco.yaml	      myvoc.yaml       VisDrone.yaml
GlobalWheat2020.yaml  Objects365.yaml  VOC.yaml


In [14]:
from detect import *
import argparse
import os
import sys
from pathlib import Path

import torch
import torch.backends.cudnn as cudnn



from models.common import DetectMultiBackend
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr, cv2,
                           increment_path, non_max_suppression, print_args, scale_coords, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, time_sync

In [15]:
@torch.no_grad()
def run(
        weights='best_155.pt',  # model.pt path(s)
        source='/ilab/datasets/local/Fall-data-large/test',  # file/dir/URL/glob, 0 for webcam
        data='./data/myvoc.yaml',  # dataset.yaml path
        imgsz=(320, 320),  # inference size (height, width)
        conf_thres=0.1,  # confidence threshold
        iou_thres=0.5,  # NMS IOU threshold
        max_det=1,  # maximum detections per image
        device='',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
        view_img=False,  # show results
        save_txt=True,  # save results to *.txt
        save_conf=False,  # save confidences in --save-txt labels
        save_crop=False,  # save cropped prediction boxes
        nosave=True,  # do not save images/videos
        classes=None,  # filter by class: --class 0, or --class 0 2 3
        agnostic_nms=False,  # class-agnostic NMS
        augment=False,  # augmented inference
        visualize=False,  # visualize features
        update=False,  # update all models
        project=ROOT / 'runs/detect',  # save results to project/name
        name='exp',  # save results to project/name
        exist_ok=False,  # existing project/name ok, do not increment
        line_thickness=3,  # bounding box thickness (pixels)
        hide_labels=False,  # hide labels
        hide_conf=False,  # hide confidences
        half=False,  # use FP16 half-precision inference
        dnn=False,  # use OpenCV DNN for ONNX inference
):
    source = str(source)
    save_img = not nosave and not source.endswith('.txt')  # save inference images
    is_file = Path(source).suffix[1:] in (IMG_FORMATS + VID_FORMATS)
    is_url = source.lower().startswith(('rtsp://', 'rtmp://', 'http://', 'https://'))
    webcam = source.isnumeric() or source.endswith('.txt') or (is_url and not is_file)
    if is_url and is_file:
        source = check_file(source)  # download

    # Directories
    save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Load model
    device = select_device(device)
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
    stride, names, pt = model.stride, model.names, model.pt
    imgsz = check_img_size(imgsz, s=stride)  # check image size

    # Dataloader
    if webcam:
        view_img = check_imshow()
        cudnn.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=imgsz, stride=stride, auto=pt)
        bs = len(dataset)  # batch_size
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt)
        bs = 1  # batch_size
    vid_path, vid_writer = [None] * bs, [None] * bs

    # Run inference
    model.warmup(imgsz=(1 if pt else bs, 3, *imgsz))  # warmup
    dt, seen = [0.0, 0.0, 0.0], 0
    for path, im, im0s, vid_cap, s in dataset:
        t=0
        t1 = time_sync()
        im = torch.from_numpy(im).to(device)
        im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
        im /= 255  # 0 - 255 to 0.0 - 1.0
        if len(im.shape) == 3:
            im = im[None]  # expand for batch dim
        t2 = time_sync()
        dt[0] += t2 - t1

        # Inference
        visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
        pred = model(im, augment=augment, visualize=visualize)
        t3 = time_sync()
        dt[1] += t3 - t2

        # NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        dt[2] += time_sync() - t3

        # Second-stage classifier (optional)
        # pred = utils.general.apply_classifier(pred, classifier_model, im, im0s)

        # Process predictions
        for i, det in enumerate(pred):  # per image
            seen += 1
            if webcam:  # batch_size >= 1
                p, im0, frame = path[i], im0s[i].copy(), dataset.count
                s += f'{i}: '
            else:
                p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # im.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # im.txt
            s += '%gx%g ' % im.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            imc = im0.copy() if save_crop else im0  # for save_crop
            annotator = Annotator(im0, line_width=line_thickness, example=str(names))
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(im.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                # Write results
                maxss = 0
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xyxy, conf) if save_conf else (cls, *xywh)  # label format
                        bbbox=[]
                        for i in xyxy:
                            bbbox.append(i.item())
                        ss = xywh[2]*xywh[3]
                        if ss*conf>maxss:
                            maxss=ss*conf
                            res = p.name+' '+namesxxx[int(cls)]+' '+str(bbbox[0])+' '+str(bbbox[1])+' '+str(bbbox[2])+' '+str(bbbox[3])
                    if save_img or save_crop or view_img:  # Add bbox to image
                        c = int(cls)  # integer class
                        label = None if hide_labels else (names[c] if hide_conf else f'{names[c]} {conf:.2f}')
                        annotator.box_label(xyxy, label, color=colors(c, True))
                    if save_crop:
                        save_one_box(xyxy, imc, file=save_dir / 'crops' / names[c] / f'{p.stem}.jpg', BGR=True)
                t=1
                df.append(res)
            # Stream results
            im0 = annotator.result()
            if view_img:
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)
                else:  # 'video' or 'stream'
                    if vid_path[i] != save_path:  # new video
                        vid_path[i] = save_path
                        if isinstance(vid_writer[i], cv2.VideoWriter):
                            vid_writer[i].release()  # release previous video writer
                        if vid_cap:  # video
                            fps = vid_cap.get(cv2.CAP_PROP_FPS)
                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        else:  # stream
                            fps, w, h = 30, im0.shape[1], im0.shape[0]
                        save_path = str(Path(save_path).with_suffix('.mp4'))  # force *.mp4 suffix on results videos
                        vid_writer[i] = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                    vid_writer[i].write(im0)

        # Print time (inference-only)
        LOGGER.info(f'{s}Done. ({t3 - t2:.3f}s)')
        if t==0:
            df.append(p.name+' stand 100.0 300.0 100.0 300.0')
    # Print results
    t = tuple(x / seen * 1E3 for x in dt)  # speeds per image
    LOGGER.info(f'Speed: %.1fms pre-process, %.1fms inference, %.1fms NMS per image at shape {(1, 3, *imgsz)}' % t)
    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        LOGGER.info(f"Results saved to {colorstr('bold', save_dir)}{s}")
    if update:
        strip_optimizer(weights)  # update model (to fix SourceChangeWarning)

In [16]:
namesxxx=['fall','stand','sit']
df=[]
run()

YOLOv5 🚀 2022-6-11 Python-3.7.6 torch-1.7.1 CPU

Fusing layers... 
YOLOv5_hat summary: 224 layers, 7059304 parameters, 0 gradients
image 1/1445 /ilab/datasets/local/Fall-data-large/test/fall-01-cam0-rgb-001.png: 256x320 1 stand, Done. (0.470s)
image 2/1445 /ilab/datasets/local/Fall-data-large/test/fall-01-cam0-rgb-011.png: 256x320 1 stand, Done. (0.395s)
image 3/1445 /ilab/datasets/local/Fall-data-large/test/fall-01-cam0-rgb-015.png: 256x320 1 stand, Done. (0.482s)
image 4/1445 /ilab/datasets/local/Fall-data-large/test/fall-01-cam0-rgb-039.png: 256x320 1 stand, Done. (0.394s)
image 5/1445 /ilab/datasets/local/Fall-data-large/test/fall-01-cam0-rgb-070.png: 256x320 1 stand, Done. (0.390s)
image 6/1445 /ilab/datasets/local/Fall-data-large/test/fall-01-cam0-rgb-074.png: 256x320 1 stand, Done. (0.486s)
image 7/1445 /ilab/datasets/local/Fall-data-large/test/fall-01-cam0-rgb-076.png: 256x320 1 stand, Done. (0.405s)
image 8/1445 /ilab/datasets/local/Fall-data-large/test/fall-01-cam0-rgb-087.pn

In [17]:
len(df)

1445

In [18]:
import pandas as pd
import csv
df=pd.DataFrame(df)
df.to_csv('/home/ilab/submission', quoting=csv.QUOTE_NONE, header=None, index=False)

In [19]:
!wc -l /home/ilab/submission

1445 /home/ilab/submission


In [21]:
cat /home/ilab/submission

fall-01-cam0-rgb-001.png fall 382.0 104.0 464.0 317.0
fall-01-cam0-rgb-011.png fall 374.0 102.0 467.0 320.0
fall-01-cam0-rgb-015.png fall 371.0 108.0 464.0 320.0
fall-01-cam0-rgb-039.png fall 378.0 115.0 465.0 319.0
fall-01-cam0-rgb-070.png fall 333.0 88.0 437.0 347.0
fall-01-cam0-rgb-074.png fall 325.0 101.0 448.0 345.0
fall-01-cam0-rgb-076.png fall 320.0 110.0 440.0 342.0
fall-01-cam0-rgb-087.png fall 254.0 96.0 411.0 382.0
fall-01-cam0-rgb-088.png fall 271.0 74.0 400.0 379.0
fall-01-cam0-rgb-093.png fall 257.0 107.0 381.0 400.0
fall-01-cam0-rgb-114.png sit 74.0 322.0 419.0 467.0
fall-01-cam0-rgb-115.png sit 138.0 318.0 442.0 480.0
fall-01-cam0-rgb-116.png sit 130.0 314.0 454.0 480.0
fall-01-cam0-rgb-126.png sit 127.0 331.0 464.0 480.0
fall-01-cam0-rgb-128.png sit 145.0 352.0 448.0 477.0
fall-01-cam0-rgb-132.png sit 204.0 365.0 464.0 476.0
fall-01-cam0-rgb-158.png sit 149.0 347.0 455.0 480.0
fall-01-cam1-rgb-096.png sit 179.0 3.0 445.0 194.0
fall-01-cam1-rgb-103.png sit 198.0 20.0 40

In [22]:
!tail /home/ilab/submission

image-92-4.jpeg fall 10.0 87.0 53.0 239.0
image-94-1.jpeg fall 223.0 34.0 320.0 240.0
image-94-2.jpeg fall 222.0 35.0 320.0 240.0
image-94-3.jpeg sit 143.0 84.0 320.0 240.0
image-94-4.jpeg fall 225.0 0.0 320.0 131.0
image-95-1.jpeg fall 223.0 0.0 320.0 159.0
image-96-2.jpeg fall 224.0 0.0 320.0 138.0
image-96-3.jpeg sit 157.0 77.0 320.0 240.0
image-96-4.jpeg fall 224.0 0.0 320.0 138.0
image-98-2.jpeg fall 161.0 0.0 284.0 181.0
